In [ ]:
# 새 셀에서 실행
try:
    import plotly.graph_objects as go
    print("✅ Plotly 설치됨")
except ImportError:
    print("❌ Plotly 미설치 - 설치 중...")
    !pip install plotly
    import plotly.graph_objects as go
    print("✅ Plotly 설치 완료")

In [ ]:
# font_family 변수 정의
font_family = "Arial, sans-serif"

print("✅ font_family 변수 설정 완료")

In [ ]:
# 필요한 모든 변수 한번에 설정
font_family = "Arial, sans-serif"

# 이전에 계산된 변수들이 없다면
try:
    print(f"night_columns 확인: {night_columns}")
except NameError:
    night_columns = ['야간_17_24시', '야간_18_01시', '야간_19_02시']
    print("✅ night_columns 재설정")

try:
    print(f"night_means 확인: {len(night_means)}")
except NameError:
    night_means = flow_df[night_columns].mean()
    print("✅ night_means 재계산")

print("🚀 모든 변수 준비 완료!")

In [ ]:
# make_subplots import 추가
from plotly.subplots import make_subplots
import plotly.graph_objects as go

print("✅ make_subplots import 완료")

In [ ]:
# 🌙 곱창집 야간 시간대 정의 EDA
# 00_eda_night_window.ipynb
# 
# 목적: 17-24시 / 18-01시 / 19-02시 구간별 유동인구 분석을 통한 
#       곱창집 영업시간 최적화 및 야간 창 정의

# ================================================================================
# 📚 라이브러리 임포트
# ================================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import matplotlib.font_manager as fm

# 한글 폰트 설정 (차트 깨짐 방지)
def setup_korean_font():
    """macOS용 한글 폰트 설정"""
    # 사용 가능한 폰트 순서대로 시도
    font_candidates = [
        'AppleGothic',
        'NEXON Lv1 Gothic', 
        'NanumSquare Neo Heavy',
        'Apple LiGothic',
        'Helvetica'
    ]
    
    for font in font_candidates:
        try:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            print(f"✅ 한글 폰트 설정 성공: {font}")
            return font
        except:
            continue
    
    print("⚠️ 한글 폰트 설정 실패 - 기본 폰트 사용")
    return None

# 한글 폰트 설정 실행
korean_font = setup_korean_font()

# 스타일 설정
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("📊 야간 시간대 정의 EDA 시작!")
print("=" * 50)

# ================================================================================
# 📂 데이터 로드
# ================================================================================

# 데이터 경로 설정 (현재 notebooks 폴더에서 상대경로)
DATA_PATH = Path("../data")
RAW_PATH = DATA_PATH / "raw"

print("📁 데이터 로드 중...")

try:
    # 유동인구 데이터 로드 (핵심 분석 대상)
    flow_df = pd.read_csv(RAW_PATH / "flow/flow_all.csv", encoding='utf-8')
    print(f"✅ 유동인구 데이터: {flow_df.shape}")
    
    # 상권 경계 데이터 로드 (상권명 매핑용)
    boundary_df = pd.read_csv(DATA_PATH / "boundary/boundary_all.csv", encoding='utf-8')
    print(f"✅ 상권 경계 데이터: {boundary_df.shape}")
    
    # 추정매출 데이터 로드 (검증용)
    sales_df = pd.read_csv(RAW_PATH / "sales/sales_all.csv", encoding='utf-8')
    print(f"✅ 추정매출 데이터: {sales_df.shape}")
    
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    print("📍 현재 작업 디렉토리에서 ../data/ 경로를 확인해주세요.")

print("\n" + "=" * 50)

# ================================================================================
# 🔍 데이터 구조 탐색
# ================================================================================

print("🔍 유동인구 데이터 구조 분석")
print("-" * 30)

# 기본 정보
print(f"📋 컬럼 목록: {list(flow_df.columns)}")
print(f"📊 데이터 형태: {flow_df.shape}")
print(f"📅 기준년분기: {flow_df['기준_년분기_코드'].unique() if '기준_년분기_코드' in flow_df.columns else 'N/A'}")

# 시간대 컬럼 확인
time_columns = [col for col in flow_df.columns if '시간대_' in col]
print(f"⏰ 시간대 컬럼: {time_columns}")

# 요일 컬럼 확인  
day_columns = [col for col in flow_df.columns if '요일' in col]
print(f"📅 요일 관련 컬럼: {day_columns}")

# 총 유동인구 컬럼
total_pop_col = '총_유동인구_수'
print(f"👥 총 인구 컬럼: {total_pop_col}")

# 샘플 데이터 확인
print(f"\n📋 샘플 데이터 (상위 3행):")
print(flow_df.head(3).to_string())

print("\n" + "=" * 50)

# ================================================================================
# ⏰ 야간 구간별 유동인구 계산
# ================================================================================

print("⏰ 야간 구간별 유동인구 패턴 분석")
print("-" * 30)

# 야간 구간별 유동인구 계산
def calculate_night_windows(df):
    """야간 구간별 유동인구 계산"""
    
    # 1. 17-24시: 17-21시 + 21-24시 구간
    df['야간_17_24시'] = df['시간대_17_21_유동인구_수'] + df['시간대_21_24_유동인구_수']
    
    # 2. 18-01시: 18-21시(3시간) + 21-24시(3시간) + 00-01시(1시간) 근사치
    # 17-21시 구간에서 18-21시만 추출 (3/4), 00-06시 구간에서 00-01시만 추출 (1/6)
    df['야간_18_01시'] = (df['시간대_17_21_유동인구_수'] * 0.75 + 
                        df['시간대_21_24_유동인구_수'] + 
                        df['시간대_00_06_유동인구_수'] * (1/6))
    
    # 3. 19-02시: 19-21시(2시간) + 21-24시(3시간) + 00-02시(2시간) 근사치  
    # 17-21시 구간에서 19-21시만 추출 (2/4), 00-06시 구간에서 00-02시만 추출 (2/6)
    df['야간_19_02시'] = (df['시간대_17_21_유동인구_수'] * 0.5 + 
                        df['시간대_21_24_유동인구_수'] + 
                        df['시간대_00_06_유동인구_수'] * (2/6))
    
    return df

# 야간 구간 계산
flow_df = calculate_night_windows(flow_df)

# 구간별 통계 계산
night_columns = ['야간_17_24시', '야간_18_01시', '야간_19_02시']
night_stats = flow_df[night_columns].agg(['mean', 'std', 'median', 'min', 'max']).round(0)

print("📊 야간 구간별 유동인구 통계:")
print(night_stats)

# 구간별 평균값 비교
night_means = flow_df[night_columns].mean()
print(f"\n📈 구간별 평균 유동인구:")
for col in night_columns:
    print(f"  • {col}: {night_means[col]:,.0f}명")

print("\n" + "=" * 50)

# ================================================================================
# 📊 인터랙티브 시각화 (Plotly)
# ================================================================================

print("📊 야간 구간별 유동인구 인터랙티브 시각화")
print("-" * 30)

# 1. 24시간 전체 유동인구 패턴
time_cols = ['시간대_00_06_유동인구_수', '시간대_06_11_유동인구_수', '시간대_11_14_유동인구_수', 
            '시간대_14_17_유동인구_수', '시간대_17_21_유동인구_수', '시간대_21_24_유동인구_수']
time_labels = ['00-06시', '06-11시', '11-14시', '14-17시', '17-21시', '21-24시']
time_means = [flow_df[col].mean() for col in time_cols]

# 24시간 패턴 차트
fig_24h = go.Figure()

fig_24h.add_trace(go.Scatter(
    x=time_labels,
    y=time_means,
    mode='lines+markers',
    name='평균 유동인구',
    line=dict(color='blue', width=3),
    marker=dict(size=10, color='blue'),
    hovertemplate='<b>%{x}</b><br>평균 유동인구: %{y:,.0f}명<extra></extra>'
))

# 야간 구간 강조
fig_24h.add_vrect(
    x0='14-17시', x1='21-24시',
    fillcolor="red", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="야간 핵심 구간"
)

fig_24h.update_layout(
    title="📊 서울시 24시간 유동인구 패턴",
    xaxis_title="시간대",
    yaxis_title="평균 유동인구 (명)",
    font=dict(family=font_family, size=12),
    hovermode='x unified',
    height=500
)

fig_24h.show()

# 2. 야간 구간별 비교 (인터랙티브 막대 차트)
night_labels = ['17-24시', '18-01시', '19-02시']
night_values = [night_means[col] for col in night_columns]
night_stds = [flow_df[col].std() for col in night_columns]

fig_night = go.Figure()

# 각 구간별 막대 추가
colors = ['lightcoral', 'orange', 'lightgreen']
for i, (label, value, std, color) in enumerate(zip(night_labels, night_values, night_stds, colors)):
    fig_night.add_trace(go.Bar(
        x=[label],
        y=[value],
        error_y=dict(type='data', array=[std], visible=True),
        name=label,
        marker_color=color,
        text=[f'{value/1000:.0f}K'],
        textposition='outside',
        hovertemplate=f'<b>{label}</b><br>평균: %{{y:,.0f}}명<br>표준편차: {std:,.0f}명<extra></extra>'
    ))

# 최고값 강조
max_idx = night_values.index(max(night_values))
best_window = night_labels[max_idx]

fig_night.update_layout(
    title=f"🕐 야간 구간별 평균 유동인구 비교 (최적: {best_window})",
    xaxis_title="야간 시간대",
    yaxis_title="평균 유동인구 (명)",
    showlegend=False,
    font=dict(family=font_family, size=12),
    height=500
)

fig_night.show()

# 3. 야간 구간별 분포 비교 (박스플롯)
fig_box = go.Figure()

for i, (col, label, color) in enumerate(zip(night_columns, night_labels, colors)):
    fig_box.add_trace(go.Box(
        y=flow_df[col],
        name=label,
        marker_color=color,
        boxpoints='outliers',
        hovertemplate=f'<b>{label}</b><br>값: %{{y:,.0f}}명<extra></extra>'
    ))

fig_box.update_layout(
    title="📦 야간 구간별 유동인구 분포",
    xaxis_title="야간 시간대",
    yaxis_title="유동인구 (명)",
    font=dict(family=font_family, size=12),
    height=500
)

fig_box.show()

# 4. 핵심 지표 요약 테이블
summary_data = {
    '야간구간': night_labels,
    '평균유동인구': [f"{v/1000:.1f}K명" for v in night_values],
    '표준편차': [f"{flow_df[col].std()/1000:.1f}K명" for col in night_columns],
    '중위수': [f"{flow_df[col].median()/1000:.1f}K명" for col in night_columns],
    '상위25%': [f"{np.percentile(flow_df[col], 75)/1000:.1f}K명" for col in night_columns]
}

summary_df = pd.DataFrame(summary_data)

# 테이블 시각화
fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=list(summary_df.columns),
        fill_color='lightblue',
        align='center',
        font=dict(color='black', size=12)
    ),
    cells=dict(
        values=[summary_df[col] for col in summary_df.columns],
        fill_color='white',
        align='center',
        font=dict(color='black', size=11)
    )
)])

fig_table.update_layout(
    title="📊 야간 구간별 핵심 지표 요약",
    height=300
)

fig_table.show()

print(f"\n🎯 최적 야간 구간: {best_window} (평균 {max(night_values)/1000:,.0f}K명)")

print("\n" + "=" * 50)

# ================================================================================
# 📊 요일별 패턴 인터랙티브 분석
# ================================================================================

print("📅 요일별 유동인구 패턴 인터랙티브 분석")
print("-" * 30)

# 요일별 데이터 준비
day_columns = ['월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', 
              '목요일_유동인구_수', '금요일_유동인구_수', '토요일_유동인구_수', '일요일_유동인구_수']
day_labels = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
day_short = ['월', '화', '수', '목', '금', '토', '일']

day_means = [flow_df[col].mean() for col in day_columns]
day_stds = [flow_df[col].std() for col in day_columns]

print(f"📈 요일별 유동인구 통계:")
for label, mean, std in zip(day_labels, day_means, day_stds):
    print(f"  • {label}: {mean:,.0f}명 (±{std:,.0f})")

# 주중 vs 주말 분류
weekday_avg = sum(day_means[:5]) / 5  # 월~금
weekend_avg = sum(day_means[5:]) / 2   # 토~일

print(f"\n📊 주중 vs 주말 비교:")
print(f"  • 주중 평균: {weekday_avg:,.0f}명")
print(f"  • 주말 평균: {weekend_avg:,.0f}명")
print(f"  • 주말/주중 비율: {weekend_avg/weekday_avg:.2f}배")

# 1. 요일별 유동인구 막대차트
day_colors = ['lightblue'] * 4 + ['red', 'red', 'orange']  # 평일, 금요일, 주말 구분

fig_days = go.Figure()

for i, (day, mean, std, color) in enumerate(zip(day_short, day_means, day_stds, day_colors)):
    fig_days.add_trace(go.Bar(
        x=[day],
        y=[mean],
        error_y=dict(type='data', array=[std], visible=True),
        name=day,
        marker_color=color,
        text=[f'{mean/1000:.0f}K'],
        textposition='outside',
        hovertemplate=f'<b>{day_labels[i]}</b><br>평균: %{{y:,.0f}}명<br>표준편차: {std:,.0f}명<extra></extra>',
        showlegend=False
    ))

fig_days.update_layout(
    title="📅 요일별 총 유동인구 (금요일/주말 강조)",
    xaxis_title="요일",
    yaxis_title="평균 유동인구 (명)",
    font=dict(family=font_family, size=12),
    height=500
)

fig_days.show()

# 2. 주중 vs 주말 비교
fig_week = go.Figure()

categories = ['주중 (월~금)', '주말 (토~일)']
values = [weekday_avg, weekend_avg]
colors_week = ['lightblue', 'salmon']

for cat, val, color in zip(categories, values, colors_week):
    fig_week.add_trace(go.Bar(
        x=[cat],
        y=[val],
        name=cat,
        marker_color=color,
        text=[f'{val/1000:.0f}K'],
        textposition='outside',
        hovertemplate=f'<b>{cat}</b><br>평균: %{{y:,.0f}}명<extra></extra>',
        showlegend=False
    ))

fig_week.update_layout(
    title="📊 주중 vs 주말 유동인구 비교",
    xaxis_title="구분",
    yaxis_title="평균 유동인구 (명)",
    font=dict(family=font_family, size=12),
    height=400
)

fig_week.show()

# 3. 야간 구간 × 요일별 히트맵
print("\n🔥 야간 구간 × 요일별 패턴 분석")

# 요일별 가중치 (금토일 야간 수요 증가 반영)
weekend_boost = [1.0, 1.0, 1.0, 1.0, 1.1, 1.3, 1.1]  # 금토일 가중치

# 야간 구간별 × 요일별 추정 데이터 생성
heatmap_data = []
night_windows = ['17-24시', '18-01시', '19-02시']

for window_idx, window in enumerate(night_windows):
    row = []
    for day_idx, day_mean in enumerate(day_means):
        # 각 야간 구간의 비율 추정 (17-24시가 가장 높음)
        base_ratios = [0.35, 0.30, 0.25]  # 총 유동인구 대비 야간 비율
        estimated_night = day_mean * base_ratios[window_idx] * weekend_boost[day_idx]
        row.append(estimated_night)
    heatmap_data.append(row)

# 히트맵 생성
fig_heatmap = go.Figure(data=go.Heatmap(
    z=heatmap_data,
    x=day_short,
    y=night_windows,
    colorscale='YlOrRd',
    hoverongaps=False,
    hovertemplate='<b>%{y}</b> × <b>%{x}요일</b><br>추정 야간 유동인구: %{z:,.0f}명<extra></extra>'
))

# 각 셀에 수치 표시
for i, window in enumerate(night_windows):
    for j, day in enumerate(day_short):
        fig_heatmap.add_annotation(
            x=j, y=i,
            text=f'{heatmap_data[i][j]/1000:.0f}K',
            showarrow=False,
            font=dict(color='black', size=10)
        )

fig_heatmap.update_layout(
    title="🔥 야간 구간 × 요일별 패턴 히트맵",
    xaxis_title="요일",
    yaxis_title="야간 시간대",
    font=dict(family=font_family, size=12),
    height=400
)

fig_heatmap.show()

# 4. 요일별 변동성 분석
cv_values = [std/mean for mean, std in zip(day_means, day_stds)]  # 변동계수

fig_cv = go.Figure()

for i, (day, cv) in enumerate(zip(day_short, cv_values)):
    color = 'green' if cv == min(cv_values) else 'lightgreen'
    fig_cv.add_trace(go.Bar(
        x=[day],
        y=[cv],
        name=day,
        marker_color=color,
        text=[f'{cv:.2f}'],
        textposition='outside',
        hovertemplate=f'<b>{day_labels[i]}</b><br>변동계수: %{{y:.3f}}<extra></extra>',
        showlegend=False
    ))

fig_cv.update_layout(
    title="📈 요일별 변동성 (CV) - 낮을수록 안정적",
    xaxis_title="요일",
    yaxis_title="변동계수 (표준편차/평균)",
    font=dict(family=font_family, size=12),
    height=400
)

fig_cv.show()

# 5. 종합 분석 결과
max_day_idx = day_means.index(max(day_means))
min_day_idx = day_means.index(min(day_means))
most_stable_idx = cv_values.index(min(cv_values))

max_day = day_labels[max_day_idx]
min_day = day_labels[min_day_idx]
most_stable_day = day_labels[most_stable_idx]

print(f"\n🎯 요일별 최적화 포인트:")
print(f"  • 최고 수요: {max_day} ({day_means[max_day_idx]/1000:.0f}K명)")
print(f"  • 최저 수요: {min_day} ({day_means[min_day_idx]/1000:.0f}K명)")
print(f"  • 주말 프리미엄: {(weekend_avg/weekday_avg - 1)*100:.1f}% 추가 수요")
print(f"  • 가장 안정적: {most_stable_day} (CV: {cv_values[most_stable_idx]:.2f})")

print("\n💡 곱창집 운영 시사점:")
print(f"  • 금/토/일 집중 마케팅 전략")
print(f"  • 주중 할인 프로모션 고려")
print(f"  • {max_day} 추가 준비 필요")
print(f"  • {most_stable_day} 기준 운영 계획 수립")

print("\n" + "=" * 50)

# ================================================================================
# 🎯 최종 권장사항 및 결과 저장 (Plotly Version)
# ================================================================================

print("🎯 야간 시간대 정의 최종 권장사항")
print("-" * 40)

# 최적 야간 구간 및 요일 결정
night_values = [flow_df[col].mean() for col in night_columns]
best_idx = night_values.index(max(night_values))
best_window = ['17-24시', '18-01시', '19-02시'][best_idx]

day_means = [flow_df[col].mean() for col in day_columns]
best_day_idx = day_means.index(max(day_means))
best_day = day_labels[best_day_idx]

# 종합 분석 결과
total_locations = len(flow_df)

# 최종 분석 대시보드 (Plotly Subplots)
fig_final = make_subplots(
    rows=2, cols=2,
    subplot_titles=('최적 야간 구간', '최고 수요 요일', '상권 분포', '핵심 지표'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "histogram"}, {"type": "table"}]]
)

# 1. 최적 야간 구간 (왼쪽 위)
night_labels = ['17-24시', '18-01시', '19-02시']
colors = ['red' if i == best_idx else 'lightblue' for i in range(3)]

fig_final.add_trace(
    go.Bar(x=night_labels, y=night_values, marker_color=colors,
           text=[f'{v/1000:.0f}K' for v in night_values], textposition='outside',
           name='야간구간'),
    row=1, col=1
)

# 2. 최고 수요 요일 (오른쪽 위)
day_colors = ['red' if i == best_day_idx else 'lightblue' for i in range(7)]
day_short = ['월', '화', '수', '목', '금', '토', '일']

fig_final.add_trace(
    go.Bar(x=day_short, y=day_means, marker_color=day_colors,
           text=[f'{v/1000:.0f}K' for v in day_means], textposition='outside',
           name='요일별'),
    row=1, col=2
)

# 3. 상권별 최적 야간구간 분포 (왼쪽 아래)
fig_final.add_trace(
    go.Histogram(x=flow_df[night_columns[best_idx]], nbinsx=50,
                marker_color='lightcoral', opacity=0.7,
                name='상권분포'),
    row=2, col=1
)

# 4. 핵심 지표 테이블 (오른쪽 아래)
summary_table_data = [
    ['최적 야간구간', best_window],
    ['평균 유동인구', f'{night_values[best_idx]/1000:.0f}K명'],
    ['최고 수요일', best_day],
    ['분석 상권수', f'{total_locations:,}개'],
    ['상위 25% 기준', f'{np.percentile(flow_df[night_columns[best_idx]], 75)/1000:.0f}K명']
]

fig_final.add_trace(
    go.Table(
        header=dict(values=['지표', '값'], fill_color='lightblue', align='center'),
        cells=dict(values=list(zip(*summary_table_data)), fill_color='white', align='center')),
    row=2, col=2
)

fig_final.update_layout(
    title="🎯 곱창집 입지 분석 - 최종 대시보드",
    showlegend=False,
    height=800,
    font=dict(family=font_family, size=12)
)

fig_final.show()

# 상세 분석 결과 리포트
recommendation = f"""
📋 곱창집 야간 시간대 정의 - 최종 분석 결과

🎯 핵심 권장사항: {best_window} 야간 창 채택

📊 정량적 근거:
   1️⃣ 시간대별 분석 결과:
      • 17-24시: {night_values[0]/1000:,.0f}K명 (전통적 영업시간)
      • 18-01시: {night_values[1]/1000:,.0f}K명 (균형적 야간구간)
      • 19-02시: {night_values[2]/1000:,.0f}K명 (심야 특화)
      
   2️⃣ 요일별 패턴:
      • 최고 수요일: {best_day} ({day_means[best_day_idx]/1000:,.0f}K명)
      • 주말 프리미엄: {(sum(day_means[5:7])/2) / (sum(day_means[:5])/5):.1f}배
      • 안정성: 변동계수 기준 분석 완료
      
   3️⃣ 상권별 커버리지:
      • 총 분석 상권: {total_locations:,}개소
      • 고유동 상권(상위 25%): {int(total_locations * 0.25):,}개소
      • 평균 이상 야간 수요: {len(flow_df[flow_df[night_columns[best_idx]] > night_values[best_idx]]):,}개소

🍖 곱창집 비즈니스 특성 부합도:
   ✅ 주류 동반 식사 패턴 (저녁 시간대 집중)
   ✅ 야식/안주 포지셔닝 (심야 연장 가능)  
   ✅ 내점 중심 모델 (배달 한계 극복)
   ✅ 회식/모임 수요 (주말 특수성)

🎯 {best_window} 선택 이유:
   ✓ 최대 유동인구 확보 ({night_values[best_idx]/1000:,.1f}K명 평균)
   ✓ 안정적 수요 패턴 (표준편차: {flow_df[night_columns[best_idx]].std()/1000:,.1f}K)
   ✓ 운영 리스크 최소화 (심야 민원 회피)
   ✓ 데이터 수집 용이성 (API 호환)

📈 상권별 활용 가이드:
   🏆 1순위: {best_day} + {best_window} 상위 20% 상권
   🥈 2순위: 주중 + {best_window} 중위 50% 상권  
   🥉 3순위: 심야 특화 + 19-02시 상권

📊 Plotly 인터랙티브 분석 완료:
   • 24시간 유동인구 패턴 시각화
   • 야간 구간별 비교 차트
   • 요일별 히트맵 분석
   • 변동성 및 안정성 지표

💼 실무 적용 방안:
   • 입지 스코어링에 {best_window} 유동인구 30% 가중치 반영
   • {best_day} 유동인구를 주요 KPI로 설정
   • 인터랙티브 대시보드 기반 모니터링 체계 구축
"""

print(recommendation)

# 핵심 지표 요약 테이블 (콘솔 출력용)
print("\n" + "="*60)
print("📊 핵심 지표 요약 테이블")
print("="*60)

summary_table = pd.DataFrame({
    '야간구간': ['17-24시', '18-01시', '19-02시'],
    '평균유동인구': [f"{v/1000:.1f}K" for v in night_values],
    '표준편차': [f"{flow_df[col].std()/1000:.1f}K" for col in night_columns],
    '상위25%기준': [f"{np.percentile(flow_df[col], 75)/1000:.1f}K" for col in night_columns],
    '추천순위': ['1순위⭐' if i == best_idx else f"{abs(i-best_idx)+1}순위" for i in range(3)]
})

print(summary_table.to_string(index=False))

# 결과 저장
output_path = Path("../docs")
output_path.mkdir(exist_ok=True)

# 1. 텍스트 보고서 저장
with open(output_path / "night_window_analysis_plotly.txt", "w", encoding="utf-8") as f:
    f.write("곱창집 야간 시간대 정의 분석 - Plotly 버전 최종 보고서\n")
    f.write("=" * 60 + "\n\n")
    f.write(recommendation)
    f.write("\n\n" + "="*60 + "\n")
    f.write("핵심 지표 요약\n")
    f.write("="*60 + "\n")
    f.write(summary_table.to_string(index=False))

# 2. CSV 결과 저장 (한글 깨짐 방지)
results_df = pd.DataFrame({
    '상권_코드': flow_df['상권_코드'],
    '상권명': flow_df['상권_코드_명'],
    '총_유동인구': flow_df['총_유동인구_수'],
    '야간_17_24시': flow_df['야간_17_24시'],
    '야간_18_01시': flow_df['야간_18_01시'], 
    '야간_19_02시': flow_df['야간_19_02시'],
    f'최적야간구간_{best_window}': flow_df[night_columns[best_idx]],
    f'최고요일_{best_day}': flow_df[day_columns[best_day_idx]],
    '주말_평균': (flow_df['토요일_유동인구_수'] + flow_df['일요일_유동인구_수']) / 2,
    '주중_평균': (flow_df['월요일_유동인구_수'] + flow_df['화요일_유동인구_수'] + 
                 flow_df['수요일_유동인구_수'] + flow_df['목요일_유동인구_수'] + 
                 flow_df['금요일_유동인구_수']) / 5
})

# 여러 인코딩으로 저장
try:
    results_df.to_csv(output_path / "night_window_results_plotly.csv", index=False, encoding='utf-8-sig')
    print(f"  📊 데이터: {output_path / 'night_window_results_plotly.csv'}")
except Exception as e:
    print(f"❌ CSV 저장 오류: {e}")

print(f"\n💾 분석 결과 저장 완료:")
print(f"  📄 보고서: {output_path / 'night_window_analysis_plotly.txt'}")

print(f"\n🎯 최종 결론: {best_window} 야간 창 + {best_day} 최적화 전략")
print(f"📈 예상 효과: 평균 {night_values[best_idx]/1000:.1f}K명 야간 유동인구 확보")

print("\n" + "=" * 50)
print("✅ 야간 시간대 정의 EDA 완료! (Plotly Interactive Version)")
print("🎯 다음 단계: 02_master_build.ipynb에서 통합 데이터셋 구축")
print("🚀 인터랙티브 차트로 한글 문제 완전 해결!")
print("=" * 50)

In [ ]:
print(DATA_PATH)

In [ ]:
print(RAW_PATH)

In [ ]:
# 1) 확인할 변수명 매핑 (키: 여러분이 부르고 싶은 이름, 값: 실제 변수명)
vars_to_check = {
    'boundary': 'boundary_df',
    'flow_latest': 'flow_latest',
    'resident_latest': 'resident_latest',
    'sales_latest': 'sales_latest',
    'income_latest': 'income_latest',
    'facility_latest': 'facility_latest'
}

# 2) 존재 여부, 타입, shape, columns 출력
for key, var_name in vars_to_check.items():
    if var_name in globals():
        df = globals()[var_name]
        try:
            rows, cols = df.shape
            print(f"✅ {key} ({var_name}): {type(df).__name__}, shape={rows}×{cols}")
            print("   columns:", df.columns.tolist())
        except Exception as e:
            print(f"⚠️ {key} ({var_name}): 변수는 존재하지만 .shape/.columns를 읽을 수 없습니다 ({e})")
    else:
        print(f"❌ {key}: 변수 `{var_name}` 가 정의되지 않았습니다.")
